In [16]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [17]:
# Descargar recursos necesarios para nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\buque\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\buque\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\buque\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
with open('best_model.pkl', 'rb') as file:
    best_model = pickle.load(file)
with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

In [19]:
def preprocess_text_multilingual(text, language):
    lemmatizer = WordNetLemmatizer()
    if language == 'spanish':
        stop_words = set(stopwords.words('spanish'))
    elif language == 'english':
        stop_words = set(stopwords.words('english'))
    else:
        # Añadir más idiomas según sea necesario
        stop_words = set()
    
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [20]:
def predict_comment(comment):
    try:
        language = detect(comment)
    except:
        language = 'unknown'
    
    if language in ['es', 'en']:  # Añadir más códigos de idiomas según sea necesario
        if language == 'es':
            language = 'spanish'
        elif language == 'en':
            language = 'english'
        preprocessed_comment = preprocess_text_multilingual(comment, language)
        vectorized_comment = vectorizer.transform([preprocessed_comment]).toarray()
        prediction = best_model.predict(vectorized_comment)
        return 'Positivo' if prediction[0] == 1 else 'Negativo'
    else:
        return 'Idioma no soportado'

In [21]:
def detect_and_preprocess(text):
    try:
        language = detect(text)
    except:
        language = 'unknown'
    
    if language in ['es', 'en']:
        if language == 'es':
            language = 'spanish'
        elif language == 'en':
            language = 'english'
        return preprocess_text_multilingual(text, language)
    else:
        return text

In [22]:
comentario_es = "Este juego es fantástico y muy divertido!"
comentario_en = "This game is fantastic and very enjoyable!"
comentario_es = "Ha sido la peor experiencia de juego de mi vida"
comentario_en = "I am not playing this game again"

resultado_es = predict_comment(comentario_es)
resultado_en = predict_comment(comentario_en)

print(f"El comentario en español: '{comentario_es}' es {resultado_es}")
print(f"El comentario en inglés: '{comentario_en}' es {resultado_en}")

El comentario en español: 'Ha sido la peor experiencia de juego de mi vida' es Negativo
El comentario en inglés: 'I am not playing this game again' es Negativo


In [23]:
comentarios_positivos_es = [
    "Este juego es increíble, me encantó cada minuto.",
    "La jugabilidad es excelente y los gráficos son impresionantes.",
    "Me encanta cómo se desarrolla la historia, es muy atrapante.",
    "Los personajes están muy bien desarrollados y la trama es genial.",
    "Muy recomendable, horas de diversión garantizada.",
    "Un juego muy bien hecho, vale cada centavo.",
    "La música y los efectos de sonido son de primera calidad.",
    "Una experiencia de juego inolvidable, lo recomiendo mucho.",
    "El mejor juego que he jugado en años, simplemente fantástico.",
    "Me sorprendió gratamente, superó todas mis expectativas."
]

comentarios_negativos_es = [
    "El juego es aburrido y repetitivo, no lo recomiendo.",
    "Los gráficos son mediocres y la jugabilidad es terrible.",
    "La historia no tiene sentido y los personajes son planos.",
    "Perdí el interés rápidamente, muy decepcionante.",
    "Un desperdicio de dinero, no vale la pena.",
    "El control es incómodo y frustrante, no pude disfrutarlo.",
    "Tiene muchos errores y bugs, arruina la experiencia.",
    "Muy corto y sin contenido, se termina en unas pocas horas.",
    "El peor juego que he jugado, una pérdida de tiempo.",
    "No cumple con lo prometido, muy decepcionante."
]

comentarios_positivos_en = [
    "This game is amazing, I loved every minute of it.",
    "The gameplay is excellent and the graphics are stunning.",
    "I love how the story unfolds, it's very engaging.",
    "The characters are well-developed and the plot is great.",
    "Highly recommended, hours of guaranteed fun.",
    "A very well-made game, worth every penny.",
    "The music and sound effects are top-notch.",
    "An unforgettable gaming experience, highly recommend it.",
    "The best game I've played in years, simply fantastic.",
    "I was pleasantly surprised, exceeded all my expectations."
]

comentarios_negativos_en = [
    "The game is boring and repetitive, I don't recommend it.",
    "The graphics are mediocre and the gameplay is terrible.",
    "The story doesn't make sense and the characters are flat.",
    "I lost interest quickly, very disappointing.",
    "A waste of money, not worth it.",
    "The controls are awkward and frustrating, couldn't enjoy it.",
    "It has many bugs and glitches, ruins the experience.",
    "Very short and lacking content, finishes in a few hours.",
    "The worst game I've played, a complete waste of time.",
    "It doesn't live up to the hype, very disappointing."
]


for comentario in comentarios_positivos_es + comentarios_negativos_es + comentarios_positivos_en + comentarios_negativos_en:
    resultado = predict_comment(comentario)
    print(f"El comentario: '{comentario}' es {resultado}")


El comentario: 'Este juego es increíble, me encantó cada minuto.' es Positivo
El comentario: 'La jugabilidad es excelente y los gráficos son impresionantes.' es Positivo
El comentario: 'Me encanta cómo se desarrolla la historia, es muy atrapante.' es Positivo
El comentario: 'Los personajes están muy bien desarrollados y la trama es genial.' es Positivo
El comentario: 'Muy recomendable, horas de diversión garantizada.' es Positivo
El comentario: 'Un juego muy bien hecho, vale cada centavo.' es Positivo
El comentario: 'La música y los efectos de sonido son de primera calidad.' es Positivo
El comentario: 'Una experiencia de juego inolvidable, lo recomiendo mucho.' es Positivo
El comentario: 'El mejor juego que he jugado en años, simplemente fantástico.' es Positivo
El comentario: 'Me sorprendió gratamente, superó todas mis expectativas.' es Positivo
El comentario: 'El juego es aburrido y repetitivo, no lo recomiendo.' es Negativo
El comentario: 'Los gráficos son mediocres y la jugabilidad

In [24]:
game_overview = pd.read_csv('game_overview.csv')

In [25]:
# Preprocesar los comentarios
game_overview = game_overview.dropna(subset=['overview'])
game_overview['overview'] = game_overview['overview'].apply(detect_and_preprocess)

In [26]:
# Vectorización del texto usando el vectorizador cargado
X_new = vectorizer.transform(game_overview['overview']).toarray()

# Evaluar el modelo en el nuevo conjunto de datos
y_new_pred = best_model.predict(X_new)

In [27]:
if 'Recomendado_binario' in game_overview.columns:
    y_true = game_overview['Recomendado_binario']
    print("Results on Game Overview Dataset:")
    print(classification_report(y_true, y_new_pred))
    print(confusion_matrix(y_true, y_new_pred))
    print(f"Accuracy: {accuracy_score(y_true, y_new_pred)}")
else:
    print("Predictions on Game Overview Dataset:")
    game_overview['prediction'] = y_new_pred
    pd.set_option('display.max_rows', None)  # Mostrar todas las filas
    print(game_overview[['overview', 'prediction']])

Predictions on Game Overview Dataset:
                                             overview  prediction
0   survive 1000 room cute terror ? break cuteness...           0
1   latest entry sakura series action-packed ever ...           1
2   free play shooter confrontation two irreconcil...           0
3   take helm gigantic capital ship get ready batt...           0
4   counter-strike : global offensive ( c : go ) e...           0
5   most-played game steam.every day , million pla...           0
6   exile , struggling survive dark continent wrae...           1
7   war thunder comprehensive free-to-play , cross...           0
8                                                               1
9   `` fun online '' - pc gameris free ! ’ catch !...           0
10  200 historic naval vessel command world warshi...           1
11                                                              1
12  take duelist around world `` yu-gi-oh ! duel l...           0
13  join 25+ million player paladin , 

In [33]:
# Función para predecir el sentimiento de un comentario independiente
def predict_comment(comment):
    preprocessed_comment = detect_and_preprocess(comment)
    vectorized_comment = vectorizer.transform([preprocessed_comment]).toarray()
    prediction = best_model.predict(vectorized_comment)
    return 'Positivo' if prediction[0] == 1 else 'Negativo'

In [34]:
comentario = "This game is fantastic and very enjoyable!"
resultado = predict_comment(comentario)
print(f"El comentario: '{comentario}' es {resultado}")

El comentario: 'This game is fantastic and very enjoyable!' es Positivo


In [35]:
comentario = "Me aburre bastante"
resultado = predict_comment(comentario)
print(f"El comentario: '{comentario}' es {resultado}")

El comentario: 'Me aburre bastante' es Negativo


In [39]:
comentario = "this game is so boring"
resultado = predict_comment(comentario)
print(f"El comentario: '{comentario}' es {resultado}")

El comentario: 'this game is so boring' es Positivo
